# TOPN

> TOPN (N_Value, Table, OrderBy_Expression, Order)

**Remarks**

If there is a tie, in Order_By values, at the N-th row of the table, then all tied rows are returned. Then, when there are ties at the N-th row the function might return more than n rows.

If N_Value is 0 (zero) or less, TOPN returns an empty table.

If Order is 0 or FALSE, sorting is DESCCENDING. If it is 1 or TRUE, it is ASCENDING.

TOPN does not guarantee any sort order for the results.

This function is not supported for use in DirectQuery mode when used in calculated columns or row-level security (RLS) rules.

Example: Ascending TOP 3 gives you the least 3 items while DESCENDING TOP 3 gives you top 3 items. However, the output will be displayed in no particular order.

    EVALUATE
        TOPN(
        3,SUMMARIZE(
            DimProduct,
            DimProduct[BrandName],
            "Sales",SUM(FactSales[SalesAmount])
            ),
        [Sales],1
        )

|BrandName|    Sales|
|---------|--------------|
|Southridge Video|486921072.3361|
|Tailspin Toys|    134428829.5546|
|Northwind Traders|418668898.7141|

## TOP3 BY Calendar Year
### Use TOPN with Generate

        EVALUATE
            GENERATE(
                SUMMARIZE(FactSales,DimDate[CalendarYear]),  
                TOPN(
                3,ADDCOLUMNS(
                    VALUES(DimProduct[BrandName]),
                    "Sales",[Sales_Amount])
                    ,
                [Sales],1
                )
                )
            ORDER BY DimDate[CalendarYear]

|CalendarYear|BrandName|Sales|
|------------|---------|-----|
|2007|Tailspin Toys|7405207.3772|
|2007|Wide World Importers|44255687.3677|
|2007|The Phone Company|43512965.86|
|2008|Southridge Video|30596284.5963|
|2008|A. Datum|37053899.646|
|2008|Tailspin Toys|7567273.9194|
|2009|Southridge Video|34929502.7205|
|2009|Tailspin Toys|15616388.7636|
|2009|Northwind Traders|21159824.3897|



## Total of Top3 by Calendar Year

> Use VAR to iterate through and aggregate subtotal by year for top3 of each year

        EVALUATE
            GENERATE(
                SUMMARIZE(FactSales,DimDate[CalendarYear]),  
                
                VAR top3=
                TOPN(
                3,ADDCOLUMNS(
                    VALUES(DimProduct[BrandName]),
                    "Sales",[Sales_Amount])
                    ,
                [Sales],1
                )
                
                VAR top3total=SUMX(top3,[Sales])
                
                RETURN {top3total}
                
                )
            ORDER BY DimDate[CalendarYear]